In [1]:
from __future__ import print_function

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.datasets import cifar100
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.layers import Conv2D,MaxPooling2D

import os
import numpy as np

In [2]:
batch_size = 32
num_classes = 100
epochs = 10
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(),"saved_models")
model_name = "keras_cifar100_trained_model.h5"

In [3]:
(train_data,train_labels),(test_data,test_labels) = cifar100.load_data()

train_data = train_data.astype("float32")
test_data = test_data.astype("float32")
train_data /= 255
test_data /= 255

In [4]:
print("train_data.shape:{}".format(train_data.shape))
print("test_data.shape:{}".format(test_data.shape))

train_data.shape:(50000, 32, 32, 3)
test_data.shape:(10000, 32, 32, 3)


In [5]:
train_labels = tf.keras.utils.to_categorical(y=train_labels,num_classes=num_classes)
test_labels = tf.keras.utils.to_categorical(y=test_labels,num_classes=num_classes)

In [6]:
input_shape = (32,32,3)

model = Sequential()
model.add(Conv2D(filters=32,kernel_size=(3,3),
                padding="same",input_shape=input_shape))
model.add(Activation(activation="relu"));
model.add(Conv2D(filters=32,kernel_size=(3,3)))
model.add(Activation(activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))


model.add(Conv2D(filters=64,kernel_size=(3,3),
                padding="same"))
model.add(Activation(activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3)))
model.add(Activation(activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))

model.add(Flatten())
model.add(Dense(units=512))
model.add(Activation(activation="relu"))
model.add(Dropout(rate=0.5))
model.add(Dense(units=num_classes))
model.add(Activation(activation="softmax"))

In [7]:
lr = 1e-4
decay = 1e-6

loss = tf.keras.losses.categorical_crossentropy
optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr,decay=decay)
metrics = ["accuracy"]

model.compile(loss=loss,
             optimizer=optimizer,
             metrics=metrics)

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 64)        1

In [9]:
if not data_augmentation:
    print("not using data augmentation")
    model.fit(train_data,train_labels,
             batch_size=batch_size,
             epochs=epochs,
             verbose=1,
             validation_data=(test_data,test_labels),
             shuffle=True)
    

else:
    print("using real-time data augmentation")
    
    datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=False,
    rotation_range=0,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.,
    zoom_range=0.,
    channel_shift_range=0.,
    fill_mode="nearest",
    cval=0,
    horizontal_flip=True,
    vertical_flip=False,
    rescale=None,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0)
    
    datagen.fit(train_data)
    
    model.fit_generator(datagen.flow(train_data,train_labels,batch_size=batch_size),
                       epochs=epochs,
                       verbose=1,
                       validation_data=(test_data,test_labels))
    
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir,model_name)
model.save(model_path)
print("saved trained model at %s " % model_path)


using real-time data augmentation
Epoch 1/10
1563/1563 [==============================] - 17s 11ms/step - loss: 4.3406 - accuracy: 0.0395 - val_loss: 4.0012 - val_accuracy: 0.0999
Epoch 2/10
1563/1563 [==============================] - 15s 10ms/step - loss: 3.9162 - accuracy: 0.0992 - val_loss: 3.6268 - val_accuracy: 0.1527
Epoch 3/10
1563/1563 [==============================] - 15s 10ms/step - loss: 3.6803 - accuracy: 0.1351 - val_loss: 3.4139 - val_accuracy: 0.1950
Epoch 4/10
1563/1563 [==============================] - 16s 10ms/step - loss: 3.5326 - accuracy: 0.1584 - val_loss: 3.2835 - val_accuracy: 0.2130
Epoch 5/10
1563/1563 [==============================] - 15s 10ms/step - loss: 3.4090 - accuracy: 0.1844 - val_loss: 3.2130 - val_accuracy: 0.2302
Epoch 6/10
1563/1563 [==============================] - 15s 10ms/step - loss: 3.3161 - accuracy: 0.2007 - val_loss: 3.0562 - val_accuracy: 0.2568
Epoch 7/10
1563/1563 [==============================] - 15s 10ms/step - loss: 3.2222 - acc

NameError: name 'score' is not defined

In [10]:
scores = model.evaluate(test_data,test_labels,verbose=1)
print("test loss:{}".format(scores[0]))
print("test acc:{}".format(scores[1]))

10000/10000 [==============================] - 1s 59us/sample - loss: 2.7769 - accuracy: 0.3102
test loss:2.7768975830078126
test acc:0.3102000057697296
